In [1]:
# NBVAL_IGNORE_OUTPUT
import lineapy

x = 100
y = 10

z = [x]
art = lineapy.save(z, "z")
pipeline_dir = lineapy.to_pipeline([art.name], framework='AIRFLOW', output_dir="~/airflow/dags")

Generated module file: ~/airflow/dags/z/z_module.py                                                                                                                 
Generated requirements file: ~/airflow/dags/z/z_requirements.txt                                                                                                    
Generated DAG file: ~/airflow/dags/z/z_dag.py                                                                                                                       
Generated Docker file: ~/airflow/dags/z/z_Dockerfile                                                                                                                


In [2]:
# NBVAL_IGNORE_OUTPUT
module_path = pipeline_dir/"z_dag.py"
print(module_path.read_text())

import z_module
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago

default_dag_args = {
    "owner": "airflow",
    "retries": 2,
    "start_date": days_ago(1),
}

with DAG(
    dag_id="z_dag",
    schedule_interval="*/15 * * * *",
    max_active_runs=1,
    catchup=False,
    default_args=default_dag_args,
) as dag:

    run_session_including_z = PythonOperator(
        task_id="run_session_including_z_task",
        python_callable=z_module.run_session_including_z,
    )



In [3]:
dag_path = pipeline_dir/"z_dag.py"
print(dag_path.read_text())

import z_module
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago

default_dag_args = {
    "owner": "airflow",
    "retries": 2,
    "start_date": days_ago(1),
}

with DAG(
    dag_id="z_dag",
    schedule_interval="*/15 * * * *",
    max_active_runs=1,
    catchup=False,
    default_args=default_dag_args,
) as dag:

    run_session_including_z = PythonOperator(
        task_id="run_session_including_z_task",
        python_callable=z_module.run_session_including_z,
    )

